In [0]:
%run ../utils/common

In [0]:

dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_subrange AS
SELECT
    lifnr AS vendor,
    ltsnr AS vendor_subrange,
    ernam AS created_by,
    erdat AS created_on,
    spras AS language_key,
    ltsbz AS vsr_description,
    minbw AS mov,
    waers AS unit
FROM (
    SELECT
        lifnr, ltsnr, ernam, erdat, spras, ltsbz, minbw, waers,
        ROW_NUMBER() OVER (
            PARTITION BY lifnr, ltsnr, ernam, erdat, spras 
            ORDER BY proc_date DESC
        ) AS rn
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm432
) x
WHERE x.rn = 1""")
